In [2]:
from cv2 import AGAST_FEATURE_DETECTOR_AGAST_5_8
from src.getter import Get
import os
from models.sqliteModels import generate_symbols


In [3]:
symbol = generate_symbols()

In [9]:
stocks = [next(symbol) for i in range(100)]


# Main Code: 

In [15]:
from src.getter import Get, stock_chunks
from src.pricehistory import price_history
from src.fundamentals import fundamental
from src.quotes import quote
from sqlalchemy import create_engine
from models.sqliteModels import generate_symbols
from models.mysql_db import insert_quote_and_fundamental_data_mysql, connection_uri, marketdata_db
import pandas as pd
import os


In [3]:
# create a Get object
get = Get()

In [4]:
# =========================================================================== #
# Get the quote and fundamental data using stock_chunks
quote_data = pd.concat([quote.data(each) for each in stock_chunks])
fundamental_data = pd.concat([fundamental.data(each)
                              for each in stock_chunks])

In [10]:
# create the marketdata database engine:
engine = connection_uri + '/' + marketdata_db
